In [152]:
import time  
import redis
from pymongo import MongoClient

In [153]:

r = redis.StrictRedis(host='bdl1.eng.tau.ac.il', port=6379)

In [154]:

client = MongoClient()
db = client['stud22']

#clean up the collection before starting the exercise
db.companies.drop()
companies = db.companies

## TODO: move setup to dedicated function

In [155]:
# The name of the Redis set
companiesSet = "company:names"
r.delete(companiesSet)
r.sadd(companiesSet, "ibm")
r.sismember(companiesSet,'ibm')

r.delete('oj')

1

In [156]:
r.sadd(companiesSet, "ibm")

0

In [157]:
print(companies.find_one({"company_name": "Chakra"}) is not None)

False


# functions

### Operation 1 - Add a new company

In [158]:
def is_company_exists(company_name):
    # check if company name in comapny set on redis
    if r.sismember(companiesSet, company_name):
        return True
    
    # if not, verify against mongo
    elif companies.find_one({"company_name": company_name}) is not None:
        return True
    
    else:
        return False
    
def add_company(company_dict):
    # getting dict for the company and insert it to db
    if 'company_name' not in company_dict:
        raise ValueError("company dict must have company_name field")
    else:
        company_name = company_dict['company_name']
    
    # verify that company name is unique
    if(is_company_exists(company_name)):
        raise ValueError("company name already taken")
        
    # and then insert to mongo and redis
    company_dict['jobs_list'] = [] # initiate jobs list
    companies.insert_one(company_dict)
    r.sadd(companiesSet, company_dict['company_name'])
    print("%s Add successfully!"%(company_name))

In [159]:
add_company({'company_name':'TAU', 'company_description':'University'})

### sainity checks
r.smembers(companiesSet)
companies.find_one({"company_name": 'TAU'})

TAU Add successfully!


{u'_id': ObjectId('624ad3a08ec14327c9e0147d'),
 u'company_description': u'University',
 u'company_name': u'TAU',
 u'jobs_list': []}

### Operation 2 - Add a new job position

In [160]:
def generate_job_id(job_title):
    # based on the hash of creation time and the job title
    return hash(str(int(time.time())) + job_title)
    
def add_job(job_dict, company_name):
    
    if not (is_company_exists(company_name)):
        raise ValueError("company doesn't exist")
    
    # generate job id and insert to the company object
    job_dict['job_id'] = generate_job_id(job_dict['job_title'])
    job_dict['application_list'] = [] # initiate application list
    companies.update_one({'company_name': company_name}, {'$push': {'jobs_list': job_dict}}, upsert = True)
    
    r.zincrby("oj","%s:%s"%(job_dict['location'],job_dict['job_title']),1)
    
    print("%s Added to %s jobs successfully!"%(job_dict['job_id'], company_name))

In [161]:
add_job({'job_title':'bi developer', 'location': 'Tel Aviv',
'requirements':['python','big data','mongodb'],
'status':'open','publish_date':'01-02-2020'},'TAU')

2719542033754339198 Added to TAU jobs successfully!


In [162]:
### sainity checks
print(companies.find_one({"company_name": 'TAU'}))
print("##########")
print(r.zrevrange('oj', 0, -1, withscores=True))

{u'jobs_list': [{u'status': u'open', u'requirements': [u'python', u'big data', u'mongodb'], u'job_id': 2719542033754339198L, u'application_list': [], u'publish_date': u'01-02-2020', u'location': u'Tel Aviv', u'job_title': u'bi developer'}], u'_id': ObjectId('624ad3a08ec14327c9e0147d'), u'company_name': u'TAU', u'company_description': u'University'}
##########
[('Tel Aviv:bi developer', 1.0)]


### Operation 3: Add a new application

- A candidate can apply when the job Status is *open*
- A candidate can apply only once to the same job
- The Candidate Email should be unique for a given candidate

In [163]:
def new_application(candidate, application_time, job_id, company_name):
    job = db.companies.find_one(
         { 
             "company_name": company_name,
             "jobs_list": 
                 { 
                   "$elemMatch": 
                       { 'job_id': job_id }
                 }
         }
    )
    
    
    if job['is_open'] == 0:
        print("sorry, this job is already taken :(")
        return -1
    if
    
    

SyntaxError: invalid syntax (<ipython-input-163-074ece7cd628>, line 17)

In [164]:
new_application({'candidate_name':'laura', 'email':'laura@gmail.com',
'linkedin':'https://www.linkedin.com/in/laura/', skills: ['python','sql']},
'01-02-2020 15:00:00', '1','TAU')

NameError: name 'new_application' is not defined

In [185]:
db.companies.find_one(
         { 
             "company_name": 'TAU',
             "jobs_list": 
                 { 
                   "$elemMatch": 
                       { 'job_id': 2719542033754339198L }
                 }
         }
    )

{u'_id': ObjectId('624ad3a08ec14327c9e0147d'),
 u'company_description': u'University',
 u'company_name': u'TAU',
 u'hello': u'world',
 u'jobs_list': [{u'1': u'world',
   u'application_list': [],
   u'job_id': 2719542033754339198L,
   u'job_title': u'bi developer',
   u'location': u'Tel Aviv',
   u'publish_date': u'01-02-2020',
   u'requirements': [u'python', u'big data', u'mongodb'],
   u'status': u'open'}]}

In [187]:
companies.update_one({ 
             "company_name": 'TAU',
             "jobs_list": 
                 { 
                   "$elemMatch": 
                       { 'job_id': 2719542033754339198L }
                 }
         }, {"$addToSet": {"application_list": "camera"}, upsert = True)

SyntaxError: invalid syntax (<ipython-input-187-e012b0898992>, line 8)